In [ ]:
#import os
# make sure pyspark tells workers to use python3 not 2 if both are installed
#os.environ['PYSPARK_PYTHON'] = '/home/ubuntu/anaconda3/bin/python3'
#os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ubuntu/anaconda3/bin/ipython'
#!pip install metapy

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql import SQLContext
import metapy
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

In [2]:
#Al ejecutar esta celda se demora un poco, así que un poco de paciencia
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local', "news_cleaned_topic_model") 
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [3]:
dfNews=spark.read.csv("file:///home/ubuntu/Caso_estudio/datasets/news.csv", inferSchema=True, header=True, encoding="UTF-8")
dfNews.show(10)


+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|              author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|  17283|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  1|  17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0|  6.0|null|After the bullet ...|
|  2|  17285|Tyrus Wong, ‘Bamb...|New York Times|        Margalit Fox|2017-01-06|2017.0|  1.0|null|When Walt Disney’...|
|  3|  17286|Among Deaths in 2...|New York Times|    William McDonald|2017-04-10|2017.0|  4.0|null|Death may be the ...|
|  4|  17287|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|  17288|Sick With a Cold,..

# Análisis Descriptivo del Dataset de Noticias

In [ ]:
dfNews.count()

In [ ]:
dfNewsFiltered=dfNews.na.drop(subset=("title","publication","author","year", "date", "month","content"))

In [ ]:
dfNewsFiltered.count()

In [ ]:
dfPublishers=dfNewsFiltered.select("publication","year", "date").toPandas()

In [ ]:
dfcountPublications=pd.value_counts(dfPublishers['publication'])

In [ ]:
dfSummary=dfcountPublications[:10]

In [ ]:
dfSummary.head(12)

In [ ]:
import nltk
import pandas as pd
import numpy as np
#import codecs
#import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer

nltk.download(['stopwords'])
from nltk.corpus import stopwords

In [ ]:
dfMainCols= dfNewsFiltered.select('id_news','title','publication','content')
dfMainCols.show(10)

In [ ]:
# Conteo de las palabras iniciales que están en el título y del contenido de la noticia
def countWords(record):
    textTitle  = record[1]
    textContent = record[3]
    textCombined = textTitle + " " + textContent
    words = textCombined.split()
    longitudTexto=len(words)
    return longitudTexto

udf_countWords = udf(countWords, IntegerType())
dfcountWords = dfMainCols.withColumn("countWords", udf_countWords(struct([dfMainCols[x] for x in dfMainCols.columns])))

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
#from pyspark.sql.functions import monotonically_increasing_id
import re

In [ ]:
#Total de palabras del título y de la descripción de los artículos
totalWords = dfcountWords.agg(F.sum("countWords")).collect()
totalWords

In [ ]:
#publishers= dfMainCols.select("publication").distinct()
#publishers.show(50)

In [ ]:
# Default list of Stopwords

stopWords1 = stopwords.words('english')

stopwords2 = [word.lower() for word in stopWords1] 

stopwords_custom = ['New', 'York','Times','Breitbart', 'News', 'Daily', 'NPR','Reuters','Guardian','CNN','Atlantic','Vox']
stopwords3 = stopwords2 + stopwords_custom

#Stemmer
ps = PorterStemmer() 

def cleanup_text(record):
    textTitle  = str(record[1])
    textPublication=str(record[2])
    textContent = str(record[3])
    
    #setStopwordsCustom =set(stopwords_custom)
    #setStopwordsCustom =setStopwordsCustom.union(set(textPublication.split()))
    #stopwords_custom1 = list(setStopwordsCustom)
    #stopwords_custom= stopwords_custom1
    #stopwords = stopwords_core + stopwords_custom1
    #title=textTitle
    #longPublication =len(textPublication) # Calcular la longitud de la cadena de caracteres  del publisher de la noticia
    #longTitle=len(textTitle)  # Calcular la longitud de la cadena del titulo de la noticia
    
    #if longTitle > longPublication:
    #    textPublisher=textTitle[longTitle-longPublication-1:longTitle] # Extrar caracteres derechos del titulo
    #    if textPublisher==textPublication:
    #        title=textTitle[0:longTitle-longPublication-1]
    #    else:
    #        title=textTitle
    
    textCombined = textTitle + " " + textContent
    # Remove special characters
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',textCombined)
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',textCombined)
    textCombined= re.sub('[^(a-zA-Z)]',' ',textCombined)
    textCombined= re.sub(' +',' ',textCombined)
    words = textCombined.split()
    tokens3 = [word.lower() for word in words if len(word)>3 and word.lower() not in stopwords3] 
    text_out = [ps.stem(w) for w in tokens3]
    
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',word) for word in words]
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',word) for word in words]
    #text_out = [re.sub('[^(a-zA-Z)]',' ',word) for word in words]                                   
    #text_out = [re.sub(' +',' ',word) for word in words] 
    # Remove stopwords and words under X length
    return text_out

In [ ]:
udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
dfCombined = dfMainCols.withColumn("CombinedTokens", udf_cleantext(struct([dfMainCols[x] for x in dfMainCols.columns])))

In [ ]:
dfCombined.show(10)

In [ ]:
def countTokens(record):
    listTokens = record[4]
    #words = textTitle.split(",")
    numTokens=len(listTokens)
    return numTokens

udf_countTokens = udf(countTokens, IntegerType())
dfcountTokens = dfCombined.withColumn("countTokens", udf_countTokens(struct([dfCombined[x] for x in dfCombined.columns])))

In [ ]:
dfcountTokens.show()

In [ ]:
totalTokens = dfcountTokens.agg(F.sum("countTokens")).collect()
totalTokens

In [ ]:
dfvocab=dfCombined.select("CombinedTokens").toPandas()

In [ ]:
vocabNews = set()
fil,col=dfvocab.shape
#print("Dimensiones de filas {} y de columnas {} de dataframe".format(fil,col))
#in workVoc.iloc[:,0]:
for i in range(fil): 
    vocabNews = vocabNews.union(set(dfvocab.iloc[i,0]))
len(vocabNews)